In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

TBATS is a forecasting method to model time series data. The main aim of this is to forecast time series with complex seasonal patterns using exponential smoothing

![tbats](https://miro.medium.com/max/1400/1*YplGt73W0QbEgNmUV9jKeQ.png)

TBATS: Trigonometric seasonality, Box-Cox transformation, ARMA errors, Trend and Seasonal components.

In order to start forecasting, we need to first install tbats package. The following steps should be implemented to create the model:

- Partition the data into two parts(say, train_data and test_data). Train_data is used to train the model and fit the model to data. The trained model is evaluated using test_data.
- Provide information about season lengths to the model (e.g., if hourly data is present, the model can be plotted weekly for all 24*7 hrs in a week).
- Fit the model to train_data using train_data to the model.
- Forecast the model ahead by a certain period of time for which you want to predict.

![tbats](https://miro.medium.com/max/1400/1*uAzCuU6lUiPQAqu9oNOl3w.png)

The final model will be chosen using Akaike information criterion (AIC).

In particular, auto ARIMA is used to decide if residuals need modeling and what p and q values are suitable.

In [ ]:
# ! pip install tbats

In [ ]:
from tbats import BATS, TBATS

In [ ]:
df = pd.read_csv('./data/serie-storiche-ecommerce-pulito-no-buchi-temporali.csv')
df.head()

In [ ]:
dataset = dataset.sort_values(by='data')
dataset = dataset[dataset.totale != 0.0]